In [ ]:
%run -p ../train_cnf_disentangle_rl.py --data cifar10 --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 8000 --test_batch_size 5000 --save ../experiments_published/cnf_conditional_disentangle_cifar10_bs8K_sratio_0_5_drop_0_5_rl_stdlearnscale_30_annealing_run3 --seed 3 --conditional True --controlled_tol True --train_mode semisup --lr 0.001 --warmup_iters 1000 --atol 1e-4  --rtol 1e-4 --weight_y 0.5 --condition_ratio 0.5 --dropout_rate 0.5 --scale_fac 1.0 --gate cnn2 --scale_std 30.0 --max_grad_norm 20.0 --annealing_std True
#

/tancode/repos/tan-ffjord/train_cnf_disentangle_rl.py
import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import lib.layers as layers
import lib.utils as utils
import lib.multiscale_parallel as multiscale_parallel
import lib.modules as modules
import lib.thops as thops

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time, count_nfe_gate
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

from tensorboardX import SummaryWriter

# go fast boi!!
torch.backends.cudnn.benchmark = True
SOLVERS = ["dopri5", "bdf", "rk4", "midpoint", 'adams', 'explicit_adams']
GATES = ["cnn1", "cnn2", "rnn"]

parser = argparse.ArgumentParser("Continuo

Files already downloaded and verified
Files already downloaded and verified


ODENVP(
  (transforms): ModuleList(
    (0): StackedCNFLayers(
      (chain): ModuleList(
        (0): LogitTransform()
        (1): CNF_Gate(
          (gate_net): FeedforwardGateII(
            (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (avg_layer): AdaptiveAvgPool2d(output_size=(1, 1))
            (linear_layer): Conv2d(10, 2, kernel_size=(1, 1), stride=(1, 1))
            (sigmoid): Sigmoid()
          )
          (odefunc): RegularizedODEfunc(
            (odefunc): ODEf

30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0


/tancode/repos/tan-ffjord/lib/layers/odefunc_rl.py:291: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(t).type_as(y)
/opt/conda/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Iter 0001 | Time 97.1083(97.1083) | Bit/dim 9.2495(9.2495) | Xent 2.3026(2.3026) | Loss 21.4998(21.4998) | Error 0.9069(0.9069) Steps 448(448.00) | Grad Norm 37.5201(37.5201) | Total Time 0.00(0.00)
Iter 0002 | Time 36.8205(95.2997) | Bit/dim 9.2477(9.2494) | Xent 2.3016(2.3026) | Loss 21.4594(21.4986) | Error 0.7760(0.9029) Steps 460(448.36) | Grad Norm 37.3531(37.5151) | Total Time 0.00(0.00)
Iter 0003 | Time 39.4216(93.6233) | Bit/dim 

...

10.598802395209582
10.598802395209582
10.598802395209582
10.598802395209582
10.598802395209582
10.598802395209582
10.598802395209582
10.598802395209582
10.598802395209582
10.598802395209582
10.598802395209582
10.598802395209582
10.598802395209582
10.598802395209582


Iter 1945 | Time 58.5484(62.0816) | Bit/dim 3.8252(3.7922) | Xent 1.0576(0.9797) | Loss 14.0662(10.7789) | Error 0.3816(0.3499) Steps 592(589.14) | Grad Norm 30.6187(13.0615) | Total Time 0.00(0.00)
Iter 1946 | Time 65.2241(62.1758) | Bit/dim 3.7876(3.7921) | Xent 0.9875(0.9799) | Loss 10.2489(10.7630) | Error 0.3549(0.3501) Steps 592(589.22) | Grad Norm 8.7941(12.9335) | Total Time 0.00(0.00)
Iter 1947 | Time 58.3492(62.0610) | Bit/dim 3.8139(3.7927) | Xent 1.0079(0.9807) | Loss 10.0745(10.7424) | Error 0.3668(0.3506) Steps 592(589.30) | Grad Norm 22.9747(13.2347) | Total Time 0.00(0.00)
Iter 1948 | Time 64.7532(62.1418) | Bit/dim 3.7911(3.7927) | Xent 0.9973(0.9812) | Loss 10.1742(10.7253) | Error 0.3622(0.3509) Steps 604(589.75) | Grad Norm 9.7178(13.1292) | Total Time 0.00(0.00)
Iter 1949 | Time 65.8248(62.2523) | Bit/dim 3.8136(3.7933) | Xent 0.9724(0.9810) | Loss 10.3034(10.7127) | Error 0.3484(0.3509) Steps 598(589.99) | Grad Norm 15.9296(13.2132) | Total Time 0.00(0.00)
Iter 19

10.53892215568862
10.53892215568862
10.53892215568862
10.53892215568862
10.53892215568862
10.53892215568862
10.53892215568862
10.53892215568862
10.53892215568862
10.53892215568862
10.53892215568862
10.53892215568862
10.53892215568862
10.53892215568862


Iter 1951 | Time 56.8534(62.0245) | Bit/dim 3.7849(3.7931) | Xent 0.9961(0.9813) | Loss 13.3947(10.7761) | Error 0.3524(0.3506) Steps 586(589.58) | Grad Norm 9.3183(13.1398) | Total Time 0.00(0.00)
Iter 1952 | Time 61.1257(61.9976) | Bit/dim 3.7997(3.7933) | Xent 0.9791(0.9812) | Loss 10.2343(10.7598) | Error 0.3496(0.3506) Steps 568(588.94) | Grad Norm 16.5463(13.2420) | Total Time 0.00(0.00)
Iter 1953 | Time 61.8219(61.9923) | Bit/dim 3.7952(3.7934) | Xent 0.9508(0.9803) | Loss 10.3168(10.7465) | Error 0.3439(0.3504) Steps 586(588.85) | Grad Norm 7.6127(13.0731) | Total Time 0.00(0.00)
Iter 1954 | Time 59.7408(61.9248) | Bit/dim 3.7929(3.7934) | Xent 0.9494(0.9794) | Loss 10.0982(10.7271) | Error 0.3426(0.3502) Steps 592(588.94) | Grad Norm 11.9645(13.0398) | Total Time 0.00(0.00)
Iter 1955 | Time 63.4716(61.9712) | Bit/dim 3.8089(3.7938) | Xent 0.9677(0.9790) | Loss 10.2233(10.7120) | Error 0.3454(0.3500) Steps 598(589.21) | Grad Norm 10.5553(12.9653) | Total Time 0.00(0.00)
Iter 19

10.479041916167665
10.479041916167665
10.479041916167665
10.479041916167665
10.479041916167665
10.479041916167665
10.479041916167665
10.479041916167665
10.479041916167665
10.479041916167665
10.479041916167665
10.479041916167665
10.479041916167665
10.479041916167665
